In [247]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import datetime
from datetime import date
import time

In [ ]:
# General Variables
today = date.today()
fdate = date.today().strftime("%m/%d/%Y")
time = time.strftime("%H:%M:%S")

# COVID 19 Informatics Report

In [104]:
# Global Data
# URL
# 1 Request URL
url = 'https://www.worldometers.info/coronavirus/'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Store the data
data = {}
#Get the table having the class country table
table = soup.find("table", attrs={"id": "main_table_countries"})
table_data = table.tbody.find_all("tr")  
# Get data into table
df = []
for i in range(0,len(table_data)):
    for td in table_data[i].find_all("td"):
        # remove any newlines and extra spaces from left and right
        df.append(td.text.replace('\n', ' ').strip())


countries = pd.DataFrame([])
for i in range(0, len(df), 9):
    countries = countries.append(pd.DataFrame({'Country/Other': df[i], 'Total Cases': df[i+1], 'New Cases':df[i+2],
                                              'Total Deaths':df[i+3], 'New Deaths':df[i+4], 'Rate 1M Pop':df[i+8]},
                                              index =[0]), ignore_index=True)

# Data Modifications and calculations
# Columns to convert to string
cols = ['Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Rate 1M Pop']
# Replace + and , for numeric values
countries = countries.applymap(lambda x: x.replace('+', ''))
countries = countries.applymap(lambda x: x.replace(',', ''))
# Convert to numeric
countries[cols] = countries[cols].apply(pd.to_numeric, errors='coerce', axis=1)
# Add mortality rate column
countries['Mor_Rate'] = round((countries['Total Deaths']/countries['Total Cases'])*100,2)
# Calculate mean mortality rate on top affected countries
df1 = countries[0:9]
mean_MR = np.mean(df1['Mor_Rate'])

The mean mortality rate for the top affected countries is: 2.52 %.


In [111]:
# US Data
# URL
# 1 Request URL
url = 'https://www.cdc.gov/coronavirus/2019-ncov/cases-in-us.html'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Extract cases data
cdc_data = soup.find_all(attrs={"class": "card-body bg-white"})
# Create dataset of extracted data
df = []
for ul in cdc_data:
    for li in ul.find_all('li'):
        df.append(li.text.replace('\n', ' ').strip())
        

# US specific cases - CDC
cases_us = df[0].split(': ')
cases_us = pd.to_numeric(cases_us[1])
# Total US deaths - CDC
deaths_us = df[1].split(': ')
deaths_us = pd.to_numeric(deaths_us[1])
# Calculate mortality rate
us_MR = round((deaths_us/cases_us)*100,2)

In [201]:
# NY State cases
# URL
# 1 Request URL
url = 'https://www.health.ny.gov/diseases/communicable/coronavirus'
page = requests.get(url)
# 2 Parse HTML content
soup = BeautifulSoup(page.text, 'html.parser')
# 3 Create a data dictionary to store the data
data = {}
#Get the table having the class country table
table = soup.find("table", attrs={"id": "case_count_table"})
table_data = table.find_all("tr")  
# Get all the headings of Lists
df = []
for i in range(0,len(table_data)):
    for td in table_data[i].find_all("td"):
        # remove any newlines and extra spaces from left and right
        df.append(td.text.replace('\n', ' ').strip())
        


counties = pd.DataFrame([])
for i in range(0, len(df), 2):
    counties = counties.append(pd.DataFrame({'County': df[i], 'Cases': df[i+1]},
                                              index =[0]), ignore_index=True)

    
# NY state Modification for Counties and State Tables
nys = counties[-3:]
counties_cases = counties[0:-3]
# Total state cases for print
cases_nys = pd.to_numeric(counties.iloc[-1,1])

# Situation

In [252]:
print('New York State currently has:', cases_nys, 'COVID-19 cases as of', time, fdate)
print('Erie County: 0 Confirmed cases of COVID-19.', '\n')

print('Kaleida: 0 Reported/Confirmed cases of COVID-19.')
print('ECMC: 0 Reported/Confirmed cases of COVID-19.')
print('Catholic Health: 0 Reported/Confirmed cases of COVID-19.')

Situation:

New York State currently has: 173 COVID-19 cases as of 23:57:53 03/10/2020
Erie County: 0 Confirmed cases of COVID-19. 

Kaleida: 0 Reported/Confirmed cases of COVID-19.
ECMC: 0 Reported/Confirmed cases of COVID-19.
Catholic Health: 0 Reported/Confirmed cases of COVID-19.


In [ ]:
print('Reported cases as of:', time, fdate)

# Background

Incubation, Exposure and Symptom Development
<br>
<br>
2-14 days represents the current official estimated incubation period range.
The median incubation period has been estimated to be 5.1 days (95% CI, 4.5 - 5.8 days).
97.5% of those who develop symptoms will do so within 11.5 days (CI, 8.2 - 15.6 days).
101 out of every 10 000 cases (99th percentile, 482) will develop symptoms after 14 days of active monitoring or quarantine.
<br>
<br>
Disease Transmission and Doubling Rate
<br>
<br>
Variable range of 3-9 days depending upon the country’s medical infrastructure, geography and population density.

In [249]:
print('Data updated:', '3/10/2020', '\n')

Data updated: 3/10/2020 



# Global

In [207]:
# Display values
print('The calculated mean mortality rate for the top affected countries is:', round(mean_MR,2),'%.')

The mean mortality rate for the top affected countries is: 2.52 %.


In [208]:
# Table of top 10 affected countries/other
df1 = countries[0:9]

fig = go.Figure(data=[go.Table(header=dict(values=['Country/Other', 'Total Cases', 'New Cases', 'Total Deaths',
                                                   'Rate per 1 Million Population', 'Mortality Rate %']),
                 cells=dict(values=[df1['Country/Other'],df1['Total Cases'], df1['New Cases'], df1['Total Deaths'],
                                   df1['Rate 1M Pop'], df1['Mor_Rate']]))
                     ])

fig.update_layout(
    title='Countries with highest cases of COVID-19')
fig.update_layout(height=425)
fig.show()

In [209]:
print('Reported cases as of:', time, fdate)

print('Sources:', '\n',
'https://www.worldometers.info/coronavirus/')

Reported cases as of: 22:14:38 10/03/2020


# US

In [210]:
# Table of cases in the US
fig = go.Figure(data=[go.Table(header=dict(values=['Total Cases', 'Total Deaths', 'Mortality Rate %']),
                 cells=dict(values=[cases_us, deaths_us, us_MR]))
                     ])

fig.update_layout(
    title='Cases of COVID-19 in the United States')
fig.update_layout(height=250)
fig.show()

In [211]:
print('Reported cases as of:', time, fdate)

Reported cases as of: 22:14:38 10/03/2020


# NYS

In [212]:
fig = go.Figure(data=[go.Table(header=dict(values=['New York State', 'Cases']),
                 cells=dict(values=[nys['County'],nys['Cases']]))
                     ])

fig.update_layout(
    title='New York State Cases of COVID-19')

fig.update_layout(height=300)
fig.show()

In [213]:
fig = go.Figure(data=[go.Table(header=dict(values=['New York State', 'Cases']),
                 cells=dict(values=[counties_cases['County'],counties_cases['Cases']]))
                     ])

fig.update_layout(
    title='New York Counties Cases of COVID-19')
fig.update_layout(height=350)
fig.show()

In [217]:
print('Reported cases as of:', time, fdate, '\n')

print('Sources:', '\n',
'https://www.health.ny.gov/diseases/communicable/coronavirus/')

Reported cases as of: 22:14:38 10/03/2020 

Sources: 
 https://www.health.ny.gov/diseases/communicable/coronavirus/


# Age, Gender and Comorbidities

In [233]:
# Mortality Rate by age categories
age_cats = ['80+ years old', '70-79 years old', '60-69 years old', '50-59 years old', '40-49 years old',
           '30-39 years old', '20-29 years old', '10-19 years old', '0-9 years old']
deaths_confirmed = ['21.9%']
deaths_all = ['14.8%', '8.0%', '3.6%', '1.3%', '0.4%', '0.2%', '0.2%', '0.2%', 'No Fatalities']

fig = go.Figure(data=[go.Table(header=dict(values=['Age', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
                 cells=dict(values=[age_cats, deaths_confirmed, deaths_all]))
                     ])

fig.update_layout(
    title='Fatality Rate by Age Groups')
fig.update_layout(height=400)
fig.show()

In [234]:
# Mortality Rate by age categories
gender = ['Male', 'Female']
deaths_confirmed = ['4.7%', '2.8%']
deaths_all = ['2.8%', '1.7%']

fig = go.Figure(data=[go.Table(header=dict(values=['Sex', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
                 cells=dict(values=[gender, deaths_confirmed, deaths_all]))
                     ])

fig.update_layout(
    title='Fatality Rate by Sex')
fig.update_layout(height=250)
fig.show()

In [235]:
# Mortality Rate by comorbidities
comor = ['Cardiovascular Disease', 'Diabetes', 'Chronic Respiratory Disease', 'Hypertension', 'Cancer',
           'No Pre-Existing Conditions']
deaths_confirmed = ['13.2%', '9.2%', '8.0%', '8.4%', '7.6%']
deaths_all = ['10.5%', '7.3%', '6.3%', '6.0%', '5.6%', '0.9%']

fig = go.Figure(data=[go.Table(header=dict(values=['Pre-Existing Condition', 'Death Rate (Confirmed Cases)', 'Death Rate(All Cases)']),
                 cells=dict(values=[comor, deaths_confirmed, deaths_all]))
                     ])

fig.update_layout(
    title='Fatality Rate by Comorbidities')
fig.update_layout(height=500)
fig.show()

In [ ]:
# Cruise Ships

In [259]:
# Create dataset
data = {'Cruise Ship':['Diamond Princess', 'World Dream', 'MS Westerdam', 'Grand Princess', 'Nile River'],
        'Date': ['2/5/20-3/1/20', '1/19/20-1/24/20', '2/1/20-2/13/20', '2/11/20-3/9/20', 'null'],
        'Location': ['Yokohoma, Japan', 'Hong Kong, China', 'Sihanoukville, Cambodia', 'Oakland, USA', 'null'],
        'Passengers': [3618, 3800, 650, 3533, 171],
        'Cases': [696, 3, 1, 21, 45],
        'Recovered': [245, None , 1, None , None],
        'Deaths': [7, None, None, 1, None]
       } 
cruise = pd.DataFrame(data)
cruise['Mortality Rate'] = round((cruise['Deaths']/cruise['Cases'])*100,2)

In [261]:
fig = go.Figure(data=[go.Table(header=dict(values=['Cruise Ship', 'Location', 'Passengers', 'Cases', 'Recovered',
                                                  'Deaths', 'Mortality Rate %']),
                 cells=dict(values=[cruise['Cruise Ship'], cruise['Location'], cruise['Passengers'],
                                   cruise['Cases'], cruise['Recovered'], cruise['Deaths'], cruise['Mortality Rate']]))
                     ])

fig.update_layout(
    title='Cruise Ships COVID-19 Cases')
fig.update_layout(height=500)
fig.show()

Sources
https://www.worldometers.info/coronavirus/coronavirus-incubation-period/
Lauer SA, Grantz KH, Bi Q, et al. The Incubation Period of Coronavirus Disease 2019 (COVID-19) From Publicly Reported Confirmed Cases: Estimation and Application. Ann Intern Med. 2020; [Epub ahead of print 10 March 2020]. doi: https://doi.org/10.7326/M20-0504
http://www.centerforhealthsecurity.org/resources/COVID-19/index.html
http://www.centerforhealthsecurity.org/resources/fact-sheets/pdfs/coronaviruses.pdf'
https://coronavirus.jhu.edu/
https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports
https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/